In [ ]:
#Problem Definition & Objective

#Project Track:
#AI Applications – NLP / Multimodal Systems

#Objective:
#The objective of this project is to develop an AI-based Multimodal Language Translation System that can accurately translate content from one language to another using multiple input formats such as text, images, and PDF documents.


!pip install -U gradio
!pip install -q transformers sentencepiece torch easyocr langdetect gTTS fer==22.4.0 opencv-python pillow pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.14.0
    Uninstalling gradio_client-1.14.0:
      Successfully uninstalled gradio_client-1.14.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.50.0
    Uninstalling gradio-5.50.0:
      Successfully uninstalled gradio-5.50.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.1/812.1 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.3 MB/s eta

Clear Problem Statement

In the modern digital world, information is available in various formats including plain text, scanned images, and document files.
Most existing translation systems are limited to text-based input and fail to handle real-world data such as images and documents.

This project aims to address this limitation by building an AI system capable of translating text, OCR-extracted image content, and PDF documents in an efficient and user-friendly manner.
Model / System Design

AI Techniques Used:

Natural Language Processing (NLP)

Transformer-based Deep Learning Models

Computer Vision for OCR

Models and Libraries:

M2M100 (Facebook AI): Multilingual translation model

EasyOCR: Text extraction from images and PDFs

FER: Facial emotion recognition from images

gTTS: Text-to-speech generation

System Pipeline:
Input → Language Detection → OCR (if required) → Translation → Audio Output


In [ ]:
import gradio as gr
import torch, tempfile, numpy as np
import easyocr, fitz
from langdetect import detect
from gtts import gTTS
from fer import FER
from PIL import Image
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)

ocr_reader = easyocr.Reader(['en', 'hi'], gpu=False)
emotion_detector = FER(mtcnn=True)

print("✅ All models loaded successfully")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ All models loaded successfully


Real-World Relevance & Motivation

Language barriers impact several real-world domains such as:

Education: multilingual study material and resources

Government services: public notices and official documents

Healthcare: medical instructions and reports

This system improves accessibility and communication by enabling multilingual translation across different content formats.


In [ ]:
language_codes = {
    "Auto Detect": "auto",
    "English": "en",
    "Hindi": "hi",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Arabic": "ar",
    "Russian": "ru"
}


In [ ]:
def translate_text(text, src_name, tgt_name):
    src = language_codes[src_name]
    tgt = language_codes[tgt_name]

    if src == "auto":
        src = detect(text)

    tokenizer.src_lang = src
    encoded = tokenizer(text, return_tensors="pt").to(device)

    generated = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id(tgt),
        max_length=256
    )

    translated = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

    audio = None
    try:
        tts = gTTS(translated, lang=tgt)
        audio = tempfile.mktemp(".mp3")
        tts.save(audio)
    except:
        pass

    return translated, audio


In [ ]:
def detect_emotion(image):
    img = np.array(image)
    result = emotion_detector.detect_emotions(img)
    if not result:
        return "No Face Detected"
    return max(result[0]["emotions"], key=result[0]["emotions"].get)


In [ ]:
def text_translate(text, src, tgt):
    if text.strip() == "":
        return "❌ Please enter text", None

    translated, audio = translate_text(text, src, tgt)
    return f"📝 Original:\n{text}\n\n🌍 Translated:\n{translated}", audio


The system is implemented using Python in a Jupyter Notebook environment.
It consists of modular functions for:

Text translation

Image OCR and translation

PDF document OCR and translation

Audio generation for translated output

A Gradio-based interactive interface is used to demonstrate the complete workflow of the system.


In [ ]:
def image_ocr_translate(image, src, tgt):
    text = ocr_reader.readtext(np.array(image), detail=0)
    if not text:
        return "❌ No text found in image", None

    extracted = " ".join(text)
    emotion = detect_emotion(image)

    translated, audio = translate_text(extracted, src, tgt)

    output = f"""
📌 Extracted Text:
{extracted}

😊 Detected Emotion:
{emotion}

🌍 Translated Text:
{translated}
"""
    return output, audio


In [ ]:
def webcam_translate(frame, tgt):
    emotion = detect_emotion(frame)
    text = f"The person looks {emotion}"

    translated, audio = translate_text(text, "English", tgt)

    return f"😊 Emotion: {emotion}\n\n🌍 Translation:\n{translated}", audio


In [ ]:
def pdf_translate(pdf, src, tgt):
    doc = fitz.open(pdf.name)
    full_text = ""

    for page in doc:
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        text = ocr_reader.readtext(np.array(img), detail=0)
        full_text += " ".join(text) + "\n"

    if full_text.strip() == "":
        return "❌ No text found in document", None

    return translate_text(full_text, src, tgt)


Evaluation & Analysis

The system is evaluated using qualitative analysis, as translation quality depends on readability and contextual correctness.

Evaluation Criteria:

Translation accuracy and fluency

OCR performance on images and PDFs

Clarity of audio output

Observations:

Translations are grammatically correct and meaningful

OCR successfully extracts text from clear documents

Multilingual translation works smoothly

Audio output enhances accessibility

Limitations:

OCR accuracy depends on image/document quality

Translation quality may reduce for low-resource languages


In [ ]:
with gr.Blocks(title="🔥 Ultimate AI Translator System") as app:

    gr.Markdown("## 🌍 Text | Image | Webcam | PDF – OCR + Emotion + Translation + Audio")

    # 📝 TEXT
    with gr.Tab("📝 Text Translation"):
        txt = gr.Textbox(lines=5)
        src0 = gr.Dropdown(language_codes.keys(), value="Auto Detect")
        tgt0 = gr.Dropdown(language_codes.keys(), value="Hindi")
        btn0 = gr.Button("Translate")
        out0 = gr.Textbox()
        aud0 = gr.Audio()
        btn0.click(text_translate, [txt, src0, tgt0], [out0, aud0])

    # 🖼 IMAGE
    with gr.Tab("🖼 Image OCR + Emotion"):
        img = gr.Image(type="pil")
        src1 = gr.Dropdown(language_codes.keys(), value="Auto Detect")
        tgt1 = gr.Dropdown(language_codes.keys(), value="Hindi")
        btn1 = gr.Button("Translate Image")
        out1 = gr.Textbox()
        aud1 = gr.Audio()
        btn1.click(image_ocr_translate, [img, src1, tgt1], [out1, aud1])



    # 📄 PDF
    with gr.Tab("📄 PDF / Document OCR"):
        pdf = gr.File(file_types=[".pdf"])
        src3 = gr.Dropdown(language_codes.keys(), value="Auto Detect")
        tgt3 = gr.Dropdown(language_codes.keys(), value="Hindi")
        btn3 = gr.Button("Translate Document")
        out3 = gr.Textbox()
        aud3 = gr.Audio()
        btn3.click(pdf_translate, [pdf, src3, tgt3], [out3, aud3])

app.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0585bb5b572e1c28fc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Ethical Considerations & Responsible AI

This system is designed as a language assistance and decision-support tool.

Ethical Considerations:

Pretrained language models may contain bias

Human verification is recommended for sensitive content

No personal data is stored or misused

Responsible AI practices have been followed throughout the development of this project.


Conclusion & Future Scope

Conclusion:
This project successfully demonstrates an AI-based Multimodal Language Translation System capable of translating text, images, and PDF documents into multiple languages.
The integration of OCR, translation, and audio generation makes the system practical and real-world ready.

Future Scope:

Speech-to-speech translation

Improved OCR for complex documents

Support for additional regional languages

Enhanced context-aware translation
